# Imports

In [1]:
import requests
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

from data_to_jupyter import time_range_query, query_to_pandas, \
                            fetch_weather_crime,call_fission_function

RUN_FROM = 'uni_wifi' #'bastion'

if RUN_FROM == 'bastion' : URL, HEADERS = 'http://fission:31001/', None
if RUN_FROM == 'uni_wifi': URL, HEADERS =  'http://172.26.135.52:9090/', {'HOST': 'fission'}

# Functions

In [2]:
def weather_to_pd(station_id: str, start_year: int, end_year: int, verb=False) -> pd.DataFrame:
    resp_dict = json.loads(requests.get(URL+f'weather/{station_id}/{start_year}/{end_year}', headers=HEADERS).text)
    data = resp_dict['Data']
    if verb : print(f'Called weather api, fetched {len(data)} lines')
    return pd.DataFrame.from_records(data)

In [3]:
def get_stream_to_pd(api: str, station_id: str, size: int, radius_km: int, verb=False) -> pd.DataFrame:
    resp_dict = json.loads(requests.get(URL+api+f'/{station_id}/{size}/{radius_km}', headers=HEADERS).text)

    count=0
    status, token, new_data = resp_dict['Status'], resp_dict['Token'], resp_dict['Data']
    data = [new_data[i]['_source'] for i in range(len(new_data))]
    if verb : print(f'Called {api} api, fetched {len(new_data)} lines')


    while (status == 200) and (new_data != []) :
        count+=1
        resp_dict = json.loads(requests.get(URL+f'stream/'+token, headers=HEADERS).text)
        status, token, new_data = resp_dict['Status'], resp_dict['Token'], resp_dict['Data']
        if verb : print(f'Called stream {count} times, fetched {len(new_data)} new lines')
        data += [new_data[i]['_source'] for i in range(len(new_data))]

    if verb: print(f'Fetched a total of {len(data)}lines')
    return pd.DataFrame.from_records(data)

# Tests

## Joined data creation

In [5]:
df_weather_full = weather_to_pd(station_id='95003', start_year =2014, end_year=2019, verb=True)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
df_weather = df_weather_full.copy()

weather_num_cols = ['UV', 'Min Temp', 'Max Temp', 'WindSpeed', 'Min Humid', 'Max Humid', 'Rain', 'Pan-Rain', 'Evapo-Rain']

for col in weather_num_cols:
    df_weather[col] = pd.to_numeric(df_weather[col])

df_weather = df_weather.rename(columns={'Date':'date'})
df_weather['date'] = pd.to_datetime(df_weather['date'], format='%d/%m/%Y').dt.date
df_weather = df_weather.drop(columns=['created_at','source', 'Station Name'])

df_weather.dtypes

In [ ]:
df_crime_full = get_stream_to_pd(api='crime', station_id='95003', size=8000, radius_km=800, verb=True)

In [ ]:
df_crime_full.head(4)

In [ ]:
df_crime = df_crime_full.groupby(['reported_date', 'description_1'])['offence_count'].sum().reset_index()

df_crime = df_crime.rename(columns={'reported_date':'date'})
df_crime['date'] = pd.to_datetime(df_crime['date']).dt.date

df_crime.dtypes

In [ ]:
df_crime

In [ ]:
df_weather

In [ ]:
df = pd.merge(df_weather, df_crime, on='date', how='inner')
df.shape

In [ ]:
df

## Linear Regression

In [ ]:
df['description_1'].unique()

### On persons

In [ ]:
df_pers = df[df['description_1'] == 'OFFENCES AGAINST THE PERSON']
df_pers = df_pers.drop(columns=['description_1', 'date'])
df_pers.head(4)

In [ ]:
df_pers.drop(columns='offence_count').values

In [ ]:
model = LinearRegression()
model.fit(df_pers.drop(columns='offence_count').values, df_pers['offence_count'])
pd.DataFrame({'Predictors': df_pers.columns[:-1], 'Coefficient': model.coef_})

In [ ]:
model = LinearRegression()
model.fit(df_pers[['Evapo-Rain']].values, df_pers['offence_count'])

# plt.scatter(df_pers[['Evapo-Rain','offence_count']].values)
plt.scatter(df_pers['Evapo-Rain'],df_pers['offence_count'])

In [ ]:
corr_table_pers = df_pers.corr(method='pearson')
corr_table_pers

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(corr_table_pers, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Correlation Matrix')
plt.show()

### On properties